# 뉴럴 네트워크

이번 강의에서 여러분은 현대 머신러닝의 시작지점인 뉴럴네트워크를 다루는 방법을 배운다.

In [5]:
# 전처리된 타이타닉 데이터셋 로드
import pandas as pd
titanic = pd.read_csv('clean_titanic.csv', index_col=0)

# 라벨 인코딩
from sklearn.preprocessing import LabelEncoder
encoders = {}
for col in titanic.columns:
    encoders[col] = LabelEncoder()
    encoders[col].fit(titanic[col])
    titanic[col] = encoders[col].transform(titanic[col])

# xs, ys 분리
xs, ys = titanic.drop(columns=['survived']), titanic['survived']

# tts
from sklearn.model_selection import train_test_split
xs_train, xs_test, ys_train, ys_test = train_test_split(xs, ys, test_size=0.2)

# 준비 끝!

# 미안하다

여러분은 이제부터 Tensor 객체를 다루는 방법을 먼저 배워야 한다.

데이터를 Tensor(행렬)로 변환하고, 조작하고 등등의 기본 작업을 수행할 줄 알아야 한다.

## Tensor 선언

Tensor(행렬)은 PyTorch의 기본 데이터 구조이다.

PyTorch는 Tensor를 사용하여 데이터를 표현하고, 연산을 수행한다.

Tensor는 다차원 배열로, NumPy의 ndarray와 유사하며, 이것으로 수치 연산을 수행하고 딥러닝 모델을 구축한다.

Tensor는 CPU와 GPU에서 모두 사용할 수 있으며, GPU를 사용하여 연산을 가속화할 수 있다.

여기서 우리는 Tensor를 선언하는 기본적인 방법을 배우고, 기존 데이터를 Tensor로 변환할 것이다.

In [51]:
from torch import Tensor

# tensor 생성 - 1, 2, 3차원
print(Tensor([1]))
print(Tensor([[1]]))
print(Tensor([[[1]]]))

tensor([1.])
tensor([[1.]])
tensor([[[1.]]])


In [53]:
print(Tensor([1]).shape)
print(Tensor([[1]]).shape)
print(Tensor([[[1]]]).shape)

torch.Size([1])
torch.Size([1, 1])
torch.Size([1, 1, 1])


In [56]:
# 2x2 tensor 생성
print(Tensor([[1, 2],
              [3, 4]]))

# 4x2 tensor 생성
print(Tensor([[1, 2],
              [3, 4],
              [5, 6],
              [7, 8]]))

tensor([[1., 2.],
        [3., 4.]])
tensor([[1., 2.],
        [3., 4.],
        [5., 6.],
        [7., 8.]])


계속 이렇게 만들어야 하나요?

**네**

In [74]:
# 빈 tensor 만들기
from torch import zeros, ones, randn

# 2x2 tensor 생성
print(zeros(2, 2))

# 4x2 tensor 생성
print(zeros(4, 2))

# 4x2x3 tensor 생성
print(zeros(4, 2, 3))

# 4x2x3x2 tensor 생성
print(zeros(4, 2, 3, 2))

tensor([[0., 0.],
        [0., 0.]])
tensor([[0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.]])
tensor([[[0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.]]])
tensor([[[[0., 0.],
          [0., 0.],
          [0., 0.]],

         [[0., 0.],
          [0., 0.],
          [0., 0.]]],


        [[[0., 0.],
          [0., 0.],
          [0., 0.]],

         [[0., 0.],
          [0., 0.],
          [0., 0.]]],


        [[[0., 0.],
          [0., 0.],
          [0., 0.]],

         [[0., 0.],
          [0., 0.],
          [0., 0.]]],


        [[[0., 0.],
          [0., 0.],
          [0., 0.]],

         [[0., 0.],
          [0., 0.],
          [0., 0.]]]])


In [78]:
# 1로 채워진 tensor 만들기
print(ones(2, 2))

# 4x2x6 tensor 생성
print(ones(4, 2, 6))

tensor([[1., 1.],
        [1., 1.]])
tensor([[[1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1.]],

        [[1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1.]],

        [[1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1.]],

        [[1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1.]]])


In [82]:
# 랜덤한 값으로 채워진 tensor 만들기
print(randn(3, 8))

tensor([[ 1.6024, -1.1606, -0.0635, -1.1592, -0.2146, -0.8592,  1.1033, -1.0854],
        [-0.1938,  1.0151, -0.5255,  1.9981, -0.1680, -0.7072, -1.7359, -0.8548],
        [ 0.2887, -0.3030,  0.6106, -0.1844,  0.3978,  1.1142,  1.5979,  2.5535]])


In [95]:
# 데이터프레임을 Tensor로 변환하기
print(type(xs_train))
print(type(xs_train.values))
print(type(Tensor(xs_train.values)))

# Tensor로 변환한 값 출력해보기
print(Tensor(xs_train.values))

<class 'pandas.core.frame.DataFrame'>
<class 'numpy.ndarray'>
<class 'torch.Tensor'>
tensor([[ 0.,  1., 60.,  ...,  1.,  2.,  1.],
        [ 1.,  0., 56.,  ...,  0.,  2.,  0.],
        [ 1.,  0., 45.,  ...,  0.,  2.,  0.],
        ...,
        [ 0.,  1., 39.,  ...,  1.,  0.,  1.],
        [ 0.,  1., 52.,  ...,  1.,  2.,  1.],
        [ 2.,  0., 39.,  ...,  0.,  0.,  0.]])


## 텐서 연산

두 텐서간의 연산을 지원한다.

In [118]:
t1 = Tensor([[1, 2],
             [3, 4]])

In [119]:
# 스칼라 덧셈, 뺄셈, 곱셈, 나눗셈
t1 + 1, t1 - 1, t1 * 2, t1 / 2

(tensor([[2., 3.],
         [4., 5.]]),
 tensor([[0., 1.],
         [2., 3.]]),
 tensor([[2., 4.],
         [6., 8.]]),
 tensor([[0.5000, 1.0000],
         [1.5000, 2.0000]]))

In [120]:
vec = Tensor([2])
t1 + vec, t1 - vec, t1 * vec, t1 / vec

(tensor([[3., 4.],
         [5., 6.]]),
 tensor([[-1.,  0.],
         [ 1.,  2.]]),
 tensor([[2., 4.],
         [6., 8.]]),
 tensor([[0.5000, 1.0000],
         [1.5000, 2.0000]]))

In [121]:
# 벡터곱
vec = Tensor([10, -10])
t1 + vec, t1-vec, t1 * vec, t1 / vec

(tensor([[11., -8.],
         [13., -6.]]),
 tensor([[-9., 12.],
         [-7., 14.]]),
 tensor([[ 10., -20.],
         [ 30., -40.]]),
 tensor([[ 0.1000, -0.2000],
         [ 0.3000, -0.4000]]))

In [130]:
# 같은 크기
t2 = Tensor([[1, -1],
             [2, -2]])

t1+t2, t1-t2, t1*t2, t1/t2

(tensor([[2., 1.],
         [5., 2.]]),
 tensor([[0., 3.],
         [1., 6.]]),
 tensor([[ 1., -2.],
         [ 6., -8.]]),
 tensor([[ 1.0000, -2.0000],
         [ 1.5000, -2.0000]]))

In [131]:
t1.matmul(t2)  # 행렬 곱셈

tensor([[  5.,  -5.],
        [ 11., -11.]])

In [134]:
# 다른 크기
t3 = Tensor([[1, 2],
             [3, 4],
             [5, 6],
             [7, 8]])

t1 + t3, t1 - t3, t1 * t3, t1 / t3  # 브로드캐스팅 실패

RuntimeError: The size of tensor a (2) must match the size of tensor b (4) at non-singleton dimension 0

## `Tensor.reshape(d, i, m, e, n, s, i, o, n)`

Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`
Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`
Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`
Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`
Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`
Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`
Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`
Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`
Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`
Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`
Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`
Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`
Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`
Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`
Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`
Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`
Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`
Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`Tensor.reshape()`
중요함

In [154]:
# reshape (재구성)
t = Tensor([[1, 2],
            [3, 4],
            [5, 6]])
print(t)

tensor([[1., 2.],
        [3., 4.],
        [5., 6.]])


In [155]:
t.flatten()  # 1차원으로 평탄화

tensor([1., 2., 3., 4., 5., 6.])

In [158]:
t.reshape(1, 2, 3)

tensor([[[1., 2., 3.],
         [4., 5., 6.]]])

In [162]:
t.reshape(-1, 3)  # 행은 되는데로, 3열

tensor([[1., 2., 3.],
        [4., 5., 6.]])

In [165]:
t.reshape(3, -1)  # 3행, 열은 되는데로

tensor([[1., 2.],
        [3., 4.],
        [5., 6.]])

In [172]:
from torch.utils.data import DataLoader, TensorDataset

# DataLoader를 사용하여 배치 단위로 데이터 로드
batch_size = 6
train_dataset = TensorDataset(Tensor(xs_train.values), Tensor(ys_train.values))
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# DataLoader를 사용하여 배치 단위로 데이터 로드
for batch_xs, batch_ys in train_loader:
    print(batch_xs)
    print(batch_ys)
    break  # 첫 번째 배치만 출력


tensor([[  2.,   1.,  28.,   0.,   0.,  61.,   2.,   1.,   1.,   2.,   1.],
        [  2.,   0.,   8.,   0.,   2., 126.,   2.,   0.,   0.,   2.,   0.],
        [  2.,   0.,  27.,   1.,   0.,  66.,   2.,   2.,   0.,   2.,   0.],
        [  2.,   0.,  13.,   1.,   1., 101.,   2.,   0.,   0.,   0.,   0.],
        [  0.,   0.,  28.,   0.,   2., 181.,   0.,   2.,   0.,   0.,   0.],
        [  0.,   1.,  31.,   0.,   0., 216.,   0.,   1.,   1.,   0.,   1.]])
tensor([0., 1., 0., 0., 1., 0.])


## Practice 1.

1. `clean_titanic` 데이터셋에서 분리한 xs_train, xs_test, ys_train, ys_test를 각각 Tensor로 변환한다.
2. xs_train, ys_train을 DataLoader에 전달하여 배치 크기 32짜리 훈련 데이터셋을 만든다.

## 네트워크 생성

이제 여러분은 Tensor를 다루는 방법을 배웠다.

이제는 뉴럴 네트워크를 생성하는 방법을 배워야 한다.

In [175]:
from torch import nn, no_grad

class TitanicModel(nn.Module):

    def __init__(self):
        super(TitanicModel, self).__init__()
        # nn.Module을 상속받아 초기화
        # 이해 못합니다 넘어가세요

        # 네트워크 정의
        self.layer = nn.Sequential(
            nn.Linear(11, 64),  # 입력층: 11개의 특성, 은닉층: 64개의 뉴런
            nn.ReLU(),
            nn.Linear(64, 32),  # 은닉층: 32개의 뉴런
            nn.ReLU(),
            nn.Linear(32, 1),   # 출력층: 1개의 뉴런 (이진 분류)
        )

        # 마지막 층 softmax
        self.softmax = nn.Sigmoid()  # 이진 분류를 위한 Sigmoid 활성화 함수


    def forward(self, x):
        # 순전파 정의
        x = self.layer(x)  # 네트워크를 통과
        x = self.softmax(x)  # 마지막 층 softmax 적용
        return x


    def predict(self, x):
        self.eval()
        with no_grad():
            return self.forward(x)  # 예측을 위한 순전파

In [176]:
# TitanicModel을 생성
model = TitanicModel()
model

TitanicModel(
  (layer): Sequential(
    (0): Linear(in_features=11, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=32, bias=True)
    (3): ReLU()
    (4): Linear(in_features=32, out_features=1, bias=True)
  )
  (softmax): Sigmoid()
)

In [183]:
# 모델의 파라메터 출력
list(model.parameters())

[Parameter containing:
 tensor([[-0.2637,  0.1568, -0.2415,  0.2464,  0.0274, -0.1780,  0.1906,  0.2192,
           0.0538, -0.2863, -0.2757],
         [-0.1447,  0.2859, -0.0147, -0.0561,  0.0224,  0.0428, -0.0447, -0.0917,
          -0.1920,  0.1950,  0.1506],
         [ 0.0796,  0.2145, -0.2267,  0.2416, -0.0918, -0.1945, -0.1859, -0.0573,
           0.1918, -0.2786,  0.1258],
         [-0.2739, -0.1167, -0.1345, -0.0328,  0.0861, -0.2848, -0.1118,  0.2124,
          -0.0283,  0.0971,  0.2708],
         [ 0.1577,  0.2182,  0.0724, -0.1025, -0.0582, -0.0548, -0.0455, -0.1537,
          -0.0790, -0.2452,  0.2216],
         [ 0.1666, -0.1492,  0.1130,  0.1228,  0.1357,  0.0158,  0.1849,  0.1490,
           0.2812,  0.0532,  0.1268],
         [-0.2645,  0.0738,  0.0456, -0.2222,  0.1941,  0.0180,  0.1727, -0.0418,
           0.3006,  0.0047, -0.1048],
         [-0.1186, -0.0204,  0.0144,  0.1956, -0.2995,  0.1439, -0.2708,  0.1886,
           0.1749,  0.1462,  0.0921],
         [-0.2095

In [185]:
# 학습 준비
from torch.optim import Adam

# 손실 함수 정의
loss_fn = nn.BCELoss()
# 이진 분류를 위한 손실 함수, BCELoss는 바이너리 크로스 엔트로피 손실이라 한다.

# 옵티마이저 정의, 이때 학습률이라는 것을 정해준다.
optimizer = Adam(model.parameters(), lr=0.001)

tensor(0.6749)